https://stackoverflow.com/questions/63528705/how-to-modify-edge-labels-in-digraph-of-networkx

In [1]:
import numpy as np
from collections import Counter
import copy

In [2]:
edges = {'in_0out_1': ['in_0', 'out_1', -3.6377842209674958],
'mid_2mid_0': ['mid_2', 'mid_0', 0.6165115214405615],
'in_1mid_1': ['in_1', 'mid_1', 3.6368075690523423],
'in_0mid_1': ['in_0', 'mid_1', -2.76795360903403],
'mid_0out_0': ['mid_0', 'out_0', -0.7665496719059972],
'mid_0out_1': ['mid_0', 'out_1', 2.0047001373416755],
'in_2mid_2': ['in_2', 'mid_2', -1.34533801312376],
'in_1mid_2': ['in_1', 'mid_2', -1.34533801312376],         
'in_mid_1_0': ['in_1', 'mid_0', 2.136059819929803],
'mid_0mid_0': ['mid_0', 'mid_0', -1.8265832443155807],
'mid_out_1_0': ['mid_1', 'out_0', 2.9330077826949488],
'in_0out_0': ['in_0', 'out_0', 2.913596825881276],
'in_1out_2': ['in_1', 'out_2', 3.5933465588280176]}

edges = [tuple(edges[i]) for i in edges]

In [3]:
d = {}
for item in edges:
    if item[1] in d:
        d[item[1]].update({item[0]:item[2]})
    else:
        d[item[1]] = {item[0]:item[2]}
        
dic = {}
for i in sorted(d):
    dic[i] = d[i]

calculate weight sum

In [4]:
def mid_to_weight(dic):
    '''weight for clear input ie. no nested weights: 0mid:{0in, 1in} or 0mid:{0mid, 0in}'''
    for key in dic:
        if 'mid' in key and isinstance(dic[key], dict):
            if set(i[:2] for i in dic[key]) == set(['in']):
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
                
            elif key in dic[key].keys() and Counter(i[:2] for i in dic[key])['mi'] == 1:
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
                
mid_to_weight(dic) 

In [5]:
def out_to_weight(dic):
    '''weight for nested input ie.  0mid:{1mid} or out:{0mid, 0in}'''
    for key in dic:
#         print('key', key)
        if 'mid' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key != key:
                    dic[key][sub_key] += dic[sub_key]

            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
                
        elif 'out' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key:
                    dic[key][sub_key] += dic[sub_key]
            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
            
out_to_weight(dic)            

In [6]:
dic

{'mid_0': -0.06476010575008019,
 'mid_1': 0.7007913892899983,
 'mid_2': -0.990838963146438,
 'out_0': 0.9999783181727204,
 'out_1': -0.9351390288575294,
 'out_2': 0.9984879701176493}